In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [2]:
path = os.getcwd()
print(path)

f:\AssignmentsUSC\Homework3


(a) Download the WPBC data from: https://archive.ics.uci.edu/ml/datasets/
Breast+Cancer+Wisconsin+(Prognostic).

In [3]:
wpbc_data = pd.read_csv(path+'/wpbc.data', header=None)

In [4]:
print(wpbc_data.head(5))#to check and verify the structure

       0  1    2      3      4       5       6        7       8       9   ...  \
0  119513  N   31  18.02  27.60  117.50  1013.0  0.09489  0.1036  0.1086  ...   
1    8423  N   61  17.99  10.38  122.80  1001.0  0.11840  0.2776  0.3001  ...   
2  842517  N  116  21.37  17.44  137.50  1373.0  0.08836  0.1189  0.1255  ...   
3  843483  N  123  11.42  20.38   77.58   386.1  0.14250  0.2839  0.2414  ...   
4  843584  R   27  20.29  14.34  135.10  1297.0  0.10030  0.1328  0.1980  ...   

       25      26      27      28      29      30      31       32   33  34  
0  139.70  1436.0  0.1195  0.1926  0.3140  0.1170  0.2677  0.08113  5.0   5  
1  184.60  2019.0  0.1622  0.6656  0.7119  0.2654  0.4601  0.11890  3.0   2  
2  159.10  1949.0  0.1188  0.3449  0.3414  0.2032  0.4334  0.09067  2.5   0  
3   98.87   567.7  0.2098  0.8663  0.6869  0.2575  0.6638  0.17300  2.0   0  
4  152.20  1575.0  0.1374  0.2050  0.4000  0.1625  0.2364  0.07678  3.5   0  

[5 rows x 35 columns]


Select the First 130 non-recurrent cases and the First 37 recurrent cases as your
training set. Add record #197 in the data set to your training set as well. (10
pts)

In [5]:
non_recurrent_cases = wpbc_data[wpbc_data[1] == 'N'].head(130)
recurrent_cases = wpbc_data[wpbc_data[1] == 'R'].head(37)

training_set = pd.concat([non_recurrent_cases, recurrent_cases])
additional_record = wpbc_data[wpbc_data[0] == 197]

if additional_record.empty or additional_record[0].values[0] not in training_set[0].values: #refered to chatgpt for cleaner code
    training_set = pd.concat([training_set, additional_record])
training_set.reset_index(drop=True, inplace=True)


(c) There are four instances in your training set that are missing the lymph node
feature (denoted as ?). This is not a very severe issue, so replace the missing
features with the median of the lymph node feature in your training set. (5 pts)

In [6]:
wpbc_data[34] = pd.to_numeric(wpbc_data[34], errors='coerce') # Convert '?' to NaN. If the error argument is passed as coerce , then invalid parsing will be set as NaN
 

In [7]:

lymph_node_median = wpbc_data[34].median()
print(lymph_node_median)
wpbc_data[34].fillna(lymph_node_median, inplace=True)

1.0


(d) Binary Classification Using Naive Bayes' Classifiers
i. Solve the problem using a Naive Bayes' classifier.3 Use Gaussian class conditional
distributions. Report the confusion matrix, ROC, precision, recall, F1
score, and AUC for both the train and test data sets. (10 pts)

In [8]:
wpbc_data.drop(columns=[0], inplace=True)#drop patient ID
print(wpbc_data)

    1    2      3      4       5       6        7        8        9        10  \
0    N   31  18.02  27.60  117.50  1013.0  0.09489  0.10360  0.10860  0.07055   
1    N   61  17.99  10.38  122.80  1001.0  0.11840  0.27760  0.30010  0.14710   
2    N  116  21.37  17.44  137.50  1373.0  0.08836  0.11890  0.12550  0.08180   
3    N  123  11.42  20.38   77.58   386.1  0.14250  0.28390  0.24140  0.10520   
4    R   27  20.29  14.34  135.10  1297.0  0.10030  0.13280  0.19800  0.10430   
..  ..  ...    ...    ...     ...     ...      ...      ...      ...      ...   
193  N   10  22.52  21.92  146.90  1597.0  0.07592  0.09162  0.06862  0.06367   
194  N    8  15.44  31.18  101.00   740.4  0.09399  0.10620  0.13750  0.06500   
195  N   12  17.17  29.19  110.00   915.3  0.08952  0.06655  0.06583  0.05068   
196  R    3  21.42  22.84  145.00  1440.0  0.10700  0.19390  0.23800  0.13180   
197  N    6  16.70  28.13  110.30   885.4  0.08896  0.11310  0.10120  0.04989   

     ...      25      26   

In [21]:
label_encoder = LabelEncoder()
wpbc_data[1] = label_encoder.fit_transform(wpbc_data[1])#following homework 1 tips to encode into labels.
X = wpbc_data.drop(columns=[1]) #features variables
y = wpbc_data[1] #outcome variable/ target
print(X.head(2))
print(y.head(2))

   2      3      4      5       6        7       8       9        10      11  \
0  31  18.02  27.60  117.5  1013.0  0.09489  0.1036  0.1086  0.07055  0.1865   
1  61  17.99  10.38  122.8  1001.0  0.11840  0.2776  0.3001  0.14710  0.2419   

   ...     25      26      27      28      29      30      31       32   33  \
0  ...  139.7  1436.0  0.1195  0.1926  0.3140  0.1170  0.2677  0.08113  5.0   
1  ...  184.6  2019.0  0.1622  0.6656  0.7119  0.2654  0.4601  0.11890  3.0   

    34  
0  5.0  
1  2.0  

[2 rows x 33 columns]
0    0
1    0
Name: 1, dtype: int64


In [10]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = GaussianNB()
clf.fit(X_train, y_train)


GaussianNB()

In [11]:
from sklearn.metrics import roc_auc_score

y_pred_train =clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_scores_train = clf.predict_proba(X_train)[:, 1]
y_scores_test = clf.predict_proba(X_test)[:, 1]

conf_matrix_train = confusion_matrix(y_train, y_pred_train)
roc_auc_train = roc_auc_score(y_train, y_scores_train)
precision_train = precision_score(y_train, y_pred_train)
recall_train = recall_score(y_train, y_pred_train)
f1_score_train = f1_score(y_train, y_pred_train)
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
roc_auc_test = roc_auc_score(y_test, y_scores_test)
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
f1_score_test = f1_score(y_test, y_pred_test)
print("Training Metrics:")
print("Confusion Matrix:\n", conf_matrix_train)
print("ROC AUC:", roc_auc_train)
print("Precision:", precision_train)
print("Recall:", recall_train)
print("F1 Score:", f1_score_train)

print("\nTesting Metrics:")
print("Confusion Matrix:\n", conf_matrix_test)
print("ROC AUC:", roc_auc_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_score_test)

Training Metrics:
Confusion Matrix:
 [[77 27]
 [12 22]]
ROC AUC: 0.7519796380090498
Precision: 0.4489795918367347
Recall: 0.6470588235294118
F1 Score: 0.5301204819277109

Testing Metrics:
Confusion Matrix:
 [[35 12]
 [ 7  6]]
ROC AUC: 0.690671031096563
Precision: 0.3333333333333333
Recall: 0.46153846153846156
F1 Score: 0.3870967741935484


In [12]:
# Metrics before and after SMOTE
test_metrics_before_SMOTE = {
    "Confusion Matrix": "[[35, 12], [7, 6]]",
    "ROC AUC": roc_auc_test,
    "Precision": precision_test,
    "Recall": recall_test,
    "F1 Score": f1_score_test
}
train_metrics_before_SMOTE = {
    "Confusion Matrix": "[[77, 27], [12, 22]]",
    "ROC AUC": roc_auc_train,
    "Precision": precision_train,
    "Recall": recall_train,
    "F1 Score": f1_score_train
}

ii. This data set is rather imbalanced. Balance your data set using SMOTE,
by downsampling the common class in the training set to 90 instances and
upsampling the uncommon class to 90 instances. Use k = 5 nearest neighbors
in SMOTE. Remember not to change the balance of the test set. Report the
confusion matrix, ROC, precision, recall, F1 score, and AUC for both the
train and test data sets. Does SMOTE help?

In [5]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter
#ref - https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/
over = SMOTE(sampling_strategy={1: 90}, k_neighbors=5, random_state=42)
under = RandomUnderSampler(sampling_strategy={0: 90}, random_state=42)
pipeline = Pipeline(steps=[('o', over), ('u', under)])
X_train_balanced, y_train_balanced = pipeline.fit_resample(X_train, y_train)
print(f"New class distribution: {Counter(y_train_balanced)}")


New class distribution: Counter({0: 90, 1: 90})


In [4]:

print("Before SMOTE:")
print(X_train.describe())
print(y_train.value_counts())
print("After SMOTE:")
print(X_train_balanced.describe())
print(y_train_balanced.value_counts())


Before SMOTE:
               2           3           4           5            6   \
count  138.000000  138.000000  138.000000  138.000000   138.000000   
mean    45.420290   17.524710   22.431014  115.626522   978.672464   
std     34.792778    3.027145    4.466822   20.555917   334.840350   
min      1.000000   10.950000   10.380000   71.900000   371.100000   
25%     13.000000   15.122500   19.517500   99.422500   716.600000   
50%     37.500000   17.495000   21.860000  114.450000   951.550000   
75%     69.500000   19.545000   25.217500  129.100000  1186.750000   
max    125.000000   25.730000   39.280000  174.200000  2010.000000   

               7           8           9           10          11  ...  \
count  138.000000  138.000000  138.000000  138.000000  138.000000  ...   
mean     0.102640    0.142777    0.156386    0.087350    0.192936  ...   
std      0.012961    0.049634    0.071569    0.033207    0.026629  ...   
min      0.078850    0.046050    0.031100    0.020310    0.

In [15]:

classifier_balanced = GaussianNB()
classifier_balanced.fit(X_train_balanced, y_train_balanced)


GaussianNB()

In [16]:
def evaluate_model(y_true, y_scores, y_pred): #used gpt's help to optimize code by using a function unlike previous step. 
    conf_matrix = confusion_matrix(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_scores)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return conf_matrix, roc_auc, precision, recall, f1


y_pred_train_balanced = classifier_balanced.predict(X_train_balanced)
y_scores_train_balanced = classifier_balanced.predict_proba(X_train_balanced)[:, 1]
metrics_train_balanced = evaluate_model(y_train_balanced, y_scores_train_balanced, y_pred_train_balanced)
y_pred_test_balanced = classifier_balanced.predict(X_test)
y_scores_test_balanced = classifier_balanced.predict_proba(X_test)[:, 1]
metrics_test_balanced = evaluate_model(y_test, y_scores_test_balanced, y_pred_test_balanced)
print("Training Metrics (Balanced):")
print("Confusion Matrix:\n", metrics_train_balanced[0])
print("ROC AUC:", metrics_train_balanced[1])
print("Precision:", metrics_train_balanced[2])
print("Recall:", metrics_train_balanced[3])
print("F1 Score:", metrics_train_balanced[4])
print("\nTesting Metrics (Balanced):")
print("Confusion Matrix:\n", metrics_test_balanced[0])
print("ROC AUC:", metrics_test_balanced[1])
print("Precision:", metrics_test_balanced[2])
print("Recall:", metrics_test_balanced[3])
print("F1 Score:", metrics_test_balanced[4])


Training Metrics (Balanced):
Confusion Matrix:
 [[66 24]
 [32 58]]
ROC AUC: 0.784320987654321
Precision: 0.7073170731707317
Recall: 0.6444444444444445
F1 Score: 0.6744186046511628

Testing Metrics (Balanced):
Confusion Matrix:
 [[35 12]
 [ 6  7]]
ROC AUC: 0.7135842880523731
Precision: 0.3684210526315789
Recall: 0.5384615384615384
F1 Score: 0.4375


In [17]:
test_metrics_after_SMOTE = {
    "Confusion Matrix": "[[35, 12], [6, 7]]",
    "ROC AUC": metrics_test_balanced[1],
    "Precision": metrics_test_balanced[2],
    "Recall": metrics_test_balanced[3],
    "F1 Score": metrics_test_balanced[4]
}
train_metrics_after_SMOTE = {
    "Confusion Matrix": "[[66, 24], [32, 58]]",
    "ROC AUC": metrics_train_balanced[1],
    "Precision": metrics_train_balanced[2],
    "Recall": metrics_train_balanced[3],
    "F1 Score": metrics_train_balanced[4]
}

In [ ]:

df_before_smote_test = pd.DataFrame(test_metrics_before_SMOTE, index=["Before SMOTE TEST"])
df_after_smote_test = pd.DataFrame(test_metrics_after_SMOTE, index=["After SMOTE TEST"])
df_before_smote_train = pd.DataFrame(train_metrics_before_SMOTE, index=["Before SMOTE train"])
df_after_smote_train = pd.DataFrame(train_metrics_after_SMOTE, index=["After SMOTE train"])

comparison_df_test= pd.concat([df_before_smote_test, df_after_smote_test])
print(comparison_df_test)
comparison_df_train= pd.concat([df_before_smote_train, df_after_smote_train])
print(comparison_df_train)



                     Confusion Matrix   ROC AUC  Precision    Recall  F1 Score
Before SMOTE TEST  [[35, 12], [7, 6]]  0.690671   0.333333  0.461538  0.387097
After SMOTE TEST   [[35, 12], [6, 7]]  0.713584   0.368421  0.538462  0.437500
                        Confusion Matrix   ROC AUC  Precision    Recall  \
Before SMOTE train  [[77, 27], [12, 22]]  0.751980   0.448980  0.647059   
After SMOTE train   [[66, 24], [32, 58]]  0.784321   0.707317  0.644444   

                    F1 Score  
Before SMOTE train  0.530120  
After SMOTE train   0.674419  


Test Set Analysis
ROC AUC: Increased from 0.690671 to 0.713584. This improvement, while not as significant as in the testing set, still shows enhanced model performance.
Precision: Improved from 0.333333 to 0.368421, suggesting a moderate increase in the accuracy of positive predictions under test conditions.
Recall: Increased from 0.461538 to 0.538462, indicating that the model is now better at identifying all positive cases.
F1 Score: Improved from 0.387097 to 0.437500, which reflects a better balance of precision and recall on the testing dataset.


Training Set Analysis
ROC AUC: Increased from 0.751980 to 0.784321. This indicates an improvement in the model's ability to distinguish between classes.
Precision: Improved significantly from 0.448980 to 0.707317, suggesting that the model became more accurate in predicting positive cases.
Recall: Decreased slightly from 0.647059 to 0.644444, a negligible change, indicating stable sensitivity.
F1 Score: Improved from 0.530120 to 0.674419, indicates a better balance between precision and recall which is a critical factor in evaluating the overall accuracy of the model.



REFERENCE:
https://stackoverflow.com/questions/34794067/how-to-set-a-cell-to-nan-in-a-pandas-dataframe

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

(e) (Extra practice, will not be graded) Solve the regression problem of estimating
time to recurrence (third attribute) using the next 32 attributes. You can use
KNN regression. To do it in a principled way, select 20% of data points each class
in your training set to choose the best k 2 1; 2; : : : ; 20, and the rest 80% as the
new training set. Report your MSE on the test set using the k you found and the
whole training set (not only the new training set!). For simplicity, use Euclidean
Distance. Repeat this process when you apply SMOTE to your new training set
to only upsample the rare class and make the data completely balanced. Does
SMOTE help in reducing the MSE?

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Split the training set for k tuning
X_tuning, X_new_train, y_tuning, y_new_train = train_test_split(X_train, y_train, test_size=0.8, random_state=42)

# Find the best k
errors = []
for k in range(1, 21):
    model = KNeighborsRegressor(n_neighbors=k)
    model.fit(X_tuning, y_tuning)
    predictions = model.predict(X_tuning)
    mse = mean_squared_error(y_tuning, predictions)
    errors.append((mse, k))

best_k = min(errors)[1]
model = KNeighborsRegressor(n_neighbors=best_k)
model.fit(X_train, y_train)  # Use the whole training set
predictions = model.predict(X_test)
mse_full = mean_squared_error(y_test, predictions)
print(f"MSE with k={best_k} on the whole training set: {mse_full}")


MSE with k=1 on the whole training set: 0.26666666666666666


In [20]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_smoted, y_smoted = smote.fit_resample(X_new_train, y_new_train)

# Retrain using the best k on the SMOTEd new training set
model.fit(X_smoted, y_smoted)
predictions_smote = model.predict(X_test)
mse_smote = mean_squared_error(y_test, predictions_smote)
print(f"MSE with SMOTE and k={best_k}: {mse_smote}")
print(f"Original MSE: {mse_full}")
print(f"MSE after SMOTE: {mse_smote}")


MSE with SMOTE and k=1: 0.36666666666666664
Original MSE: 0.26666666666666666
MSE after SMOTE: 0.36666666666666664


the final test answer reveals that MSE before and after remains same and hence SMOTE here does not help.